In [ ]:
import os
import collections
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import cv2

from sklearn import preprocessing 


import xml.etree.ElementTree as ET

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import SequentialSampler
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
from torch.nn.parallel import DataParallel
from torch.nn.parallel import DistributedDataParallel as DDP
import copy
import math
from PIL import Image
import cv2
import albumentations as A  
import pandas as pd
from tqdm import tqdm 

from sklearn import preprocessing
import matplotlib.pyplot as plt

In [ ]:
data_path = '/kaggle/input/pokemondetection-ssd/PokemonDetection_SSD/PokemonDetection_SSD/PascalVoc_chinhthuc'
df_train = pd.read_csv('/kaggle/input/pokemondetection-ssd/PokemonDetection_SSD/PokemonDetection_SSD/PascalVoc_chinhthuc/train_labels.csv')
df_test = pd.read_csv('/kaggle/input/pokemondetection-ssd/PokemonDetection_SSD/PokemonDetection_SSD/PascalVoc_chinhthuc/test_labels.csv')

In [ ]:
model_filepath = '/kaggle/input/d/huydob/model-epoch250/model-epoch250.pt'

if type(model_filepath) == str:
    assert os.path.exists(model_filepath) 
else:
    assert model_filepath is None

In [ ]:
df_train['name'].value_counts()

In [ ]:
df_test['name'].value_counts()

In [ ]:
enc = preprocessing.LabelEncoder()
df_train['labels'] = enc.fit_transform(df_train['name']) + 1
df_test['labels'] = enc.transform(df_test['name'])+1

In [ ]:
classes = df_train[['name','labels']].value_counts()
classes

In [ ]:
class_test = df_test[['name','labels']].value_counts()
class_test

In [ ]:
classes= {1:'Cinderace',2:'Dracovish',3:'Dragonite',4:'Eevee',5:'Eternatus',6:'Gengar',7:'Grookey',8:'Inteleon',9:'Lucario',10:'Meowth',11:'Mew',12:'Mimey',13:'Morpeko',14:'Pikachu',15:'Sirfetch\'d',16:'Wobbuffet',17:'Yamper',18:'Zacian',19:'Zamazenta'}

In [ ]:
df_train.drop(columns=['name'], inplace=True)
df_train.head()

In [ ]:
df_train

In [ ]:
df_test.drop(columns=['name'], inplace=True)
df_test.head()

In [ ]:
df_test

In [ ]:
print('train: ',len(df_train['filename'].unique()))
print('test: ',len(df_test['filename'].unique()))

In [ ]:
image_ids_train = df_train['filename'].unique()
image_ids_test = df_test['filename'].unique()
print("train",len(image_ids_train))
print("test",len(image_ids_test))

In [ ]:
train_df = df_train[df_train['filename'].isin(image_ids_train)]
test_df = df_test[df_test['filename'].isin(image_ids_test)]
print(train_df.shape)
print(test_df.shape)

In [ ]:
class VOCDataset(Dataset):
    
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()
        
        self.image_ids = dataframe['filename'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['filename'] == image_id]
        
        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        rows, cols = image.shape[:2]
        
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        
       
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        label = records['labels'].values
        labels = torch.as_tensor(label, dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1,0)
            
            return image, target
        
    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
# image_ids = train_df['filename'].unique()
# image_id = image_ids[1]
# print(image_id)
# records = train_df[train_df['filename'] == image_id]
# print(records)
# boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
# area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
# area = torch.as_tensor(area, dtype=torch.float32)
# print(boxes)
# print(area)
# label = records['labels'].values
# labels = torch.as_tensor(label, dtype=torch.int64)
# print(labels)

In [ ]:
# image = cv2.imread(f'/kaggle/input/pokemondetection-ssd/PokemonDetection_SSD/PokemonDetection_SSD/PascalVoc_chinhthuc/images/{image_id}', cv2.IMREAD_COLOR)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)

In [ ]:
# iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
# print(iscrowd)
# target = {}
# target['boxes'] = boxes
# target['labels'] = labels
# # target['masks'] = None
# target['image_id'] = torch.tensor([1])
# target['area'] = area
# target['iscrowd'] = iscrowd
# print(target)
        

In [ ]:
def get_transform_train():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        ToTensorV2(p=1.0)
    ], bbox_params={'format':'pascal_voc', 'label_fields': ['labels']})

def get_transform_valid():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields':['labels']})

In [ ]:
IMG_PATH = '/kaggle/input/pokemondetection-ssd/PokemonDetection_SSD/PokemonDetection_SSD/PascalVoc_chinhthuc/images'
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = VOCDataset(train_df, IMG_PATH , get_transform_train())
valid_dataset = VOCDataset(test_df, IMG_PATH, get_transform_valid())

# split the dataset in train and test set
# indices = torch.randperm(len(train_dataset)).tolist()


train_data_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
images, targets= next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
plt.figure(figsize=(20,20))
for i, (image, target) in enumerate(zip(images, targets)):
    plt.subplot(2,2, i+1)
    boxes = targets[i]['boxes'].cpu().numpy().astype(np.int32)
    sample = images[i].permute(1,2,0).cpu().numpy()
    names = targets[i]['labels'].cpu().numpy().astype(np.int64)
    for i,box in enumerate(boxes):
        cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (0, 0, 220), 2)
        cv2.putText(sample, classes[names[i]], (box[0],box[1]+15),cv2.FONT_HERSHEY_COMPLEX ,0.5,(255,0,0),3,cv2.LINE_AA)  

    plt.axis('off')
    plt.imshow(sample)

In [ ]:
# ! pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# ! git clone https://github.com/pytorch/vision.git
# ! cd vision;cp references/detection/utils.py ../;cp references/detection/transforms.py ../;cp references/detection/coco_eval.py ../;cp references/detection/engine.py ../;cp references/detection/coco_utils.py ../

In [ ]:
if model_filepath is None: 
    model = models.detection.fasterrcnn_resnet50_fpn_v2(pretrained=True)
    num_classes = 20  
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model = model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.1, weight_decay=0.0005, momentum = 0.9)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
else:
    assert os.path.exists(model_filepath)
    model = torch.load(model_filepath)    
    model = model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.0001, weight_decay=0.0005, momentum = 0.9)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [ ]:
# def find_free_port():
#     """ https://stackoverflow.com/questions/1365265/on-localhost-how-do-i-pick-a-free-port-number """
#     import socket
#     from contextlib import closing

#     with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
#         s.bind(('', 0))
#         s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
#         return str(s.getsockname()[1])
# free_port = find_free_port()    
# print(free_port)

In [ ]:
# torch.cuda.device_count()

In [ ]:
# import torch.distributed as dist
# import os
# import torch.multiprocessing as mp
# from torch.utils.data.distributed import DistributedSampler
# from torch.nn.parallel import DistributedDataParallel as DDP
# from torch.distributed import init_process_group, destroy_process_group

# def ddp_setup(rank: int, world_size: int):
#     os.environ["MASTER_ADDR"] = "localhost"
#     os.environ["MASTER_PORT"] = free_port
#     torch.cuda.set_device(rank)
#     init_process_group(backend="nccl", rank=rank, world_size=world_size)

# Train

In [ ]:
! git clone https://github.com/pytorch/vision.git
! cd vision;cp references/detection/utils.py ../;cp references/detection/transforms.py ../;cp references/detection/coco_eval.py ../;cp references/detection/engine.py ../;cp references/detection/coco_utils.py ../

In [ ]:
# from engine import train_one_epoch, evaluate
import utils


In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    for images, targets in metric_logger.log_every(tqdm(data_loader), print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger

In [ ]:
# def train_one_epoch(model, optimizer, loader, device, epoch):
#     model.to(device)
#     model.train()
    
# #     lr_scheduler = None
# #     if epoch == 0:
# #         warmup_factor = 1.0 / 1000 # do lr warmup
# #         warmup_iters = min(1000, len(loader) - 1)
        
# #         lr_scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor = warmup_factor, total_iters=warmup_iters)
    
#     all_losses = []
#     all_losses_dict = []
    
#     for images, targets in tqdm(loader):
#         images = list(image.to(device) for image in images)
#         targets = [{k: torch.tensor(v).to(device) for k, v in t.items()} for t in targets]
        
#         loss_dict = model(images, targets) # the model computes the loss automatically if we pass in targets
#         losses = sum(loss for loss in loss_dict.values())
#         loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
#         loss_value = losses.item()
        
#         all_losses.append(loss_value)
#         all_losses_dict.append(loss_dict_append)
        
#         if not math.isfinite(loss_value):
#             print(f"Loss is {loss_value}, stopping trainig") # train if loss becomes infinity
#             print(loss_dict)
#             sys.exit(1)
        
#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step()
        
# #         if lr_scheduler is not None:
# #             lr_scheduler.step() # 
                
#     all_losses_dict = pd.DataFrame(all_losses_dict) # for printing
#     print("Epoch {}, lr: {:.6f}, loss: {:.6f}, loss_classifier: {:.6f}, loss_box: {:.6f}, loss_rpn_box: {:.6f}, loss_object: {:.6f}".format(
#         epoch, optimizer.param_groups[0]['lr'], np.mean(all_losses),
#         all_losses_dict['loss_classifier'].mean(),
#         all_losses_dict['loss_box_reg'].mean(),
#         all_losses_dict['loss_rpn_box_reg'].mean(),
#         all_losses_dict['loss_objectness'].mean()
#     ))
#     if epoch % 99 == 1:
#         model_filepath = f'model-epoch{epoch + 1:0>3d}.pt'
#         torch.save(model, model_filepath)
#         print('Model saved to ', model_filepath)

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    # train for one epoch, printing every 100 iterations
    train_one_epoch(model, optimizer, train_data_loader, device, epoch, print_freq = 288)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
#     evaluate(model, valid_data_loader, device=device)

    if epoch % 10 == 0:
        model_filepath = f'model-epoch{epoch + 1:0>3d}.pt'
        torch.save(model, model_filepath)
        print('Model saved to ', model_filepath)

# Eval

In [ ]:
!pip install scikit-learn --upgrade --user

In [ ]:
!pip install pycocotools

In [ ]:
# from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [ ]:
# !cat /opt/conda/lib/python3.10/site-packages/pycocotools/cocoeval.py

In [ ]:
# update_numpy = '''
# __author__ = 'tsungyi'

# import numpy as np
# import datetime
# import time
# from collections import defaultdict
# from . import mask as maskUtils
# import copy

# class COCOeval:
#     # Interface for evaluating detection on the Microsoft COCO dataset.
#     #
#     # The usage for CocoEval is as follows:
#     #  cocoGt=..., cocoDt=...       # load dataset and results
#     #  E = CocoEval(cocoGt,cocoDt); # initialize CocoEval object
#     #  E.params.recThrs = ...;      # set parameters as desired
#     #  E.evaluate();                # run per image evaluation
#     #  E.accumulate();              # accumulate per image results
#     #  E.summarize();               # display summary metrics of results
#     # For example usage see evalDemo.m and http://mscoco.org/.
#     #
#     # The evaluation parameters are as follows (defaults in brackets):
#     #  imgIds     - [all] N img ids to use for evaluation
#     #  catIds     - [all] K cat ids to use for evaluation
#     #  iouThrs    - [.5:.05:.95] T=10 IoU thresholds for evaluation
#     #  recThrs    - [0:.01:1] R=101 recall thresholds for evaluation
#     #  areaRng    - [...] A=4 object area ranges for evaluation
#     #  maxDets    - [1 10 100] M=3 thresholds on max detections per image
#     #  iouType    - ['segm'] set iouType to 'segm', 'bbox' or 'keypoints'
#     #  iouType replaced the now DEPRECATED useSegm parameter.
#     #  useCats    - [1] if true use category labels for evaluation
#     # Note: if useCats=0 category labels are ignored as in proposal scoring.
#     # Note: multiple areaRngs [Ax2] and maxDets [Mx1] can be specified.
#     #
#     # evaluate(): evaluates detections on every image and every category and
#     # concats the results into the "evalImgs" with fields:
#     #  dtIds      - [1xD] id for each of the D detections (dt)
#     #  gtIds      - [1xG] id for each of the G ground truths (gt)
#     #  dtMatches  - [TxD] matching gt id at each IoU or 0
#     #  gtMatches  - [TxG] matching dt id at each IoU or 0
#     #  dtScores   - [1xD] confidence of each dt
#     #  gtIgnore   - [1xG] ignore flag for each gt
#     #  dtIgnore   - [TxD] ignore flag for each dt at each IoU
#     #
#     # accumulate(): accumulates the per-image, per-category evaluation
#     # results in "evalImgs" into the dictionary "eval" with fields:
#     #  params     - parameters used for evaluation
#     #  date       - date evaluation was performed
#     #  counts     - [T,R,K,A,M] parameter dimensions (see above)
#     #  precision  - [TxRxKxAxM] precision for every evaluation setting
#     #  recall     - [TxKxAxM] max recall for every evaluation setting
#     # Note: precision and recall==-1 for settings with no gt objects.
#     #
#     # See also coco, mask, pycocoDemo, pycocoEvalDemo
#     #
#     # Microsoft COCO Toolbox.      version 2.0
#     # Data, paper, and tutorials available at:  http://mscoco.org/
#     # Code written by Piotr Dollar and Tsung-Yi Lin, 2015.
#     # Licensed under the Simplified BSD License [see coco/license.txt]
#     def __init__(self, cocoGt=None, cocoDt=None, iouType='segm'):
#         \'''
#         Initialize CocoEval using coco APIs for gt and dt
#         :param cocoGt: coco object with ground truth annotations
#         :param cocoDt: coco object with detection results
#         :return: None
#         \'''
#         if not iouType:
#             print('iouType not specified. use default iouType segm')
#         self.cocoGt   = cocoGt              # ground truth COCO API
#         self.cocoDt   = cocoDt              # detections COCO API
#         self.evalImgs = defaultdict(list)   # per-image per-category evaluation results [KxAxI] elements
#         self.eval     = {}                  # accumulated evaluation results
#         self._gts = defaultdict(list)       # gt for evaluation
#         self._dts = defaultdict(list)       # dt for evaluation
#         self.params = Params(iouType=iouType) # parameters
#         self._paramsEval = {}               # parameters for evaluation
#         self.stats = []                     # result summarization
#         self.ious = {}                      # ious between all gts and dts
#         if not cocoGt is None:
#             self.params.imgIds = sorted(cocoGt.getImgIds())
#             self.params.catIds = sorted(cocoGt.getCatIds())


#     def _prepare(self):
#         \'''
#         Prepare ._gts and ._dts for evaluation based on params
#         :return: None
#         \'''
#         def _toMask(anns, coco):
#             # modify ann['segmentation'] by reference
#             for ann in anns:
#                 rle = coco.annToRLE(ann)
#                 ann['segmentation'] = rle
#         p = self.params
#         if p.useCats:
#             gts=self.cocoGt.loadAnns(self.cocoGt.getAnnIds(imgIds=p.imgIds, catIds=p.catIds))
#             dts=self.cocoDt.loadAnns(self.cocoDt.getAnnIds(imgIds=p.imgIds, catIds=p.catIds))
#         else:
#             gts=self.cocoGt.loadAnns(self.cocoGt.getAnnIds(imgIds=p.imgIds))
#             dts=self.cocoDt.loadAnns(self.cocoDt.getAnnIds(imgIds=p.imgIds))

#         # convert ground truth to mask if iouType == 'segm'
#         if p.iouType == 'segm':
#             _toMask(gts, self.cocoGt)
#             _toMask(dts, self.cocoDt)
#         # set ignore flag
#         for gt in gts:
#             gt['ignore'] = gt['ignore'] if 'ignore' in gt else 0
#             gt['ignore'] = 'iscrowd' in gt and gt['iscrowd']
#             if p.iouType == 'keypoints':
#                 gt['ignore'] = (gt['num_keypoints'] == 0) or gt['ignore']
#         self._gts = defaultdict(list)       # gt for evaluation
#         self._dts = defaultdict(list)       # dt for evaluation
#         for gt in gts:
#             self._gts[gt['image_id'], gt['category_id']].append(gt)
#         for dt in dts:
#             self._dts[dt['image_id'], dt['category_id']].append(dt)
#         self.evalImgs = defaultdict(list)   # per-image per-category evaluation results
#         self.eval     = {}                  # accumulated evaluation results

#     def evaluate(self):
#        \'''
#         Run per image evaluation on given images and store results (a list of dict) in self.evalImgs
#         :return: None
#         \'''
#         tic = time.time()
#         print('Running per image evaluation...')
#         p = self.params
#         # add backward compatibility if useSegm is specified in params
#         if not p.useSegm is None:
#             p.iouType = 'segm' if p.useSegm == 1 else 'bbox'
#             print('useSegm (deprecated) is not None. Running {} evaluation'.format(p.iouType))
#         print('Evaluate annotation type *{}*'.format(p.iouType))
#         p.imgIds = list(np.unique(p.imgIds))
#         if p.useCats:
#             p.catIds = list(np.unique(p.catIds))
#         p.maxDets = sorted(p.maxDets)
#         self.params=p

#         self._prepare()
#         # loop through images, area range, max detection number
#         catIds = p.catIds if p.useCats else [-1]

#         if p.iouType == 'segm' or p.iouType == 'bbox':
#             computeIoU = self.computeIoU
#         elif p.iouType == 'keypoints':
#             computeIoU = self.computeOks
#         self.ious = {(imgId, catId): computeIoU(imgId, catId) \
#                         for imgId in p.imgIds
#                         for catId in catIds}

#         evaluateImg = self.evaluateImg
#         maxDet = p.maxDets[-1]
#         self.evalImgs = [evaluateImg(imgId, catId, areaRng, maxDet)
#                  for catId in catIds
#                  for areaRng in p.areaRng
#                  for imgId in p.imgIds
#              ]
#         self._paramsEval = copy.deepcopy(self.params)
#         toc = time.time()
#         print('DONE (t={:0.2f}s).'.format(toc-tic))

#     def computeIoU(self, imgId, catId):
#         p = self.params
#         if p.useCats:
#             gt = self._gts[imgId,catId]
#             dt = self._dts[imgId,catId]
#         else:
#             gt = [_ for cId in p.catIds for _ in self._gts[imgId,cId]]
#             dt = [_ for cId in p.catIds for _ in self._dts[imgId,cId]]
#         if len(gt) == 0 and len(dt) ==0:
#             return []
#         inds = np.argsort([-d['score'] for d in dt], kind='mergesort')
#         dt = [dt[i] for i in inds]
#         if len(dt) > p.maxDets[-1]:
#             dt=dt[0:p.maxDets[-1]]

#         if p.iouType == 'segm':
#             g = [g['segmentation'] for g in gt]
#             d = [d['segmentation'] for d in dt]
#         elif p.iouType == 'bbox':
#             g = [g['bbox'] for g in gt]
#             d = [d['bbox'] for d in dt]
#         else:
#             raise Exception('unknown iouType for iou computation')

#         # compute iou between each dt and gt region
#         iscrowd = [int(o['iscrowd']) for o in gt]
#         ious = maskUtils.iou(d,g,iscrowd)
#         return ious

#     def computeOks(self, imgId, catId):
#         p = self.params
#         # dimention here should be Nxm
#         gts = self._gts[imgId, catId]
#         dts = self._dts[imgId, catId]
#         inds = np.argsort([-d['score'] for d in dts], kind='mergesort')
#         dts = [dts[i] for i in inds]
#         if len(dts) > p.maxDets[-1]:
#             dts = dts[0:p.maxDets[-1]]
#         # if len(gts) == 0 and len(dts) == 0:
#         if len(gts) == 0 or len(dts) == 0:
#             return []
#         ious = np.zeros((len(dts), len(gts)))
#         sigmas = p.kpt_oks_sigmas
#         vars = (sigmas * 2)**2
#         k = len(sigmas)
#         # compute oks between each detection and ground truth object
#         for j, gt in enumerate(gts):
#             # create bounds for ignore regions(double the gt bbox)
#             g = np.array(gt['keypoints'])
#             xg = g[0::3]; yg = g[1::3]; vg = g[2::3]
#             k1 = np.count_nonzero(vg > 0)
#             bb = gt['bbox']
#             x0 = bb[0] - bb[2]; x1 = bb[0] + bb[2] * 2
#             y0 = bb[1] - bb[3]; y1 = bb[1] + bb[3] * 2
#             for i, dt in enumerate(dts):
#                 d = np.array(dt['keypoints'])
#                 xd = d[0::3]; yd = d[1::3]
#                 if k1>0:
#                     # measure the per-keypoint distance if keypoints visible
#                     dx = xd - xg
#                     dy = yd - yg
#                 else:
#                     # measure minimum distance to keypoints in (x0,y0) & (x1,y1)
#                     z = np.zeros((k))
#                     dx = np.max((z, x0-xd),axis=0)+np.max((z, xd-x1),axis=0)
#                     dy = np.max((z, y0-yd),axis=0)+np.max((z, yd-y1),axis=0)
#                 e = (dx**2 + dy**2) / vars / (gt['area']+np.spacing(1)) / 2
#                 if k1 > 0:
#                     e=e[vg > 0]
#                 ious[i, j] = np.sum(np.exp(-e)) / e.shape[0]
#         return ious

#     def evaluateImg(self, imgId, catId, aRng, maxDet):
#         \'''
#         perform evaluation for single category and image
#         :return: dict (single image results)
#         \'''
#         p = self.params
#         if p.useCats:
#             gt = self._gts[imgId,catId]
#             dt = self._dts[imgId,catId]
#         else:
#             gt = [_ for cId in p.catIds for _ in self._gts[imgId,cId]]
#             dt = [_ for cId in p.catIds for _ in self._dts[imgId,cId]]
#         if len(gt) == 0 and len(dt) ==0:
#             return None

#         for g in gt:
#             if g['ignore'] or (g['area']<aRng[0] or g['area']>aRng[1]):
#                 g['_ignore'] = 1
#             else:
#                 g['_ignore'] = 0

#         # sort dt highest score first, sort gt ignore last
#         gtind = np.argsort([g['_ignore'] for g in gt], kind='mergesort')
#         gt = [gt[i] for i in gtind]
#         dtind = np.argsort([-d['score'] for d in dt], kind='mergesort')
#         dt = [dt[i] for i in dtind[0:maxDet]]
#         iscrowd = [int(o['iscrowd']) for o in gt]
#         # load computed ious
#         ious = self.ious[imgId, catId][:, gtind] if len(self.ious[imgId, catId]) > 0 else self.ious[imgId, catId]

#         T = len(p.iouThrs)
#         G = len(gt)
#         D = len(dt)
#         gtm  = np.zeros((T,G))
#         dtm  = np.zeros((T,D))
#         gtIg = np.array([g['_ignore'] for g in gt])
#         dtIg = np.zeros((T,D))
#         if not len(ious)==0:
#             for tind, t in enumerate(p.iouThrs):
#                 for dind, d in enumerate(dt):
#                     # information about best match so far (m=-1 -> unmatched)
#                     iou = min([t,1-1e-10])
#                     m   = -1
#                     for gind, g in enumerate(gt):
#                         # if this gt already matched, and not a crowd, continue
#                         if gtm[tind,gind]>0 and not iscrowd[gind]:
#                             continue
#                         # if dt matched to reg gt, and on ignore gt, stop
#                         if m>-1 and gtIg[m]==0 and gtIg[gind]==1:
#                             break
#                         # continue to next gt unless better match made
#                         if ious[dind,gind] < iou:
#                             continue
#                         # if match successful and best so far, store appropriately
#                         iou=ious[dind,gind]
#                         m=gind
#                     # if match made store id of match for both dt and gt
#                     if m ==-1:
#                         continue
#                     dtIg[tind,dind] = gtIg[m]
#                     dtm[tind,dind]  = gt[m]['id']
#                     gtm[tind,m]     = d['id']
#         # set unmatched detections outside of area range to ignore
#         a = np.array([d['area']<aRng[0] or d['area']>aRng[1] for d in dt]).reshape((1, len(dt)))
#         dtIg = np.logical_or(dtIg, np.logical_and(dtm==0, np.repeat(a,T,0)))
#         # store results for given image and category
#         return {
#                 'image_id':     imgId,
#                 'category_id':  catId,
#                 'aRng':         aRng,
#                 'maxDet':       maxDet,
#                 'dtIds':        [d['id'] for d in dt],
#                 'gtIds':        [g['id'] for g in gt],
#                 'dtMatches':    dtm,
#                 'gtMatches':    gtm,
#                 'dtScores':     [d['score'] for d in dt],
#                 'gtIgnore':     gtIg,
#                 'dtIgnore':     dtIg,
#             }

#     def accumulate(self, p = None):
#         \'''
#         Accumulate per image evaluation results and store the result in self.eval
#         :param p: input params for evaluation
#         :return: None
#         \'''
#         print('Accumulating evaluation results...')
#         tic = time.time()
#         if not self.evalImgs:
#             print('Please run evaluate() first')
#         # allows input customized parameters
#         if p is None:
#             p = self.params
#         p.catIds = p.catIds if p.useCats == 1 else [-1]
#         T           = len(p.iouThrs)
#         R           = len(p.recThrs)
#         K           = len(p.catIds) if p.useCats else 1
#         A           = len(p.areaRng)
#         M           = len(p.maxDets)
#         precision   = -np.ones((T,R,K,A,M)) # -1 for the precision of absent categories
#         recall      = -np.ones((T,K,A,M))
#         scores      = -np.ones((T,R,K,A,M))

#         # create dictionary for future indexing
#         _pe = self._paramsEval
#         catIds = _pe.catIds if _pe.useCats else [-1]
#         setK = set(catIds)
#         setA = set(map(tuple, _pe.areaRng))
#         setM = set(_pe.maxDets)
#         setI = set(_pe.imgIds)
#         # get inds to evaluate
#         k_list = [n for n, k in enumerate(p.catIds)  if k in setK]
#         m_list = [m for n, m in enumerate(p.maxDets) if m in setM]
#         a_list = [n for n, a in enumerate(map(lambda x: tuple(x), p.areaRng)) if a in setA]
#         i_list = [n for n, i in enumerate(p.imgIds)  if i in setI]
#         I0 = len(_pe.imgIds)
#         A0 = len(_pe.areaRng)
#         # retrieve E at each category, area range, and max number of detections
#         for k, k0 in enumerate(k_list):
#             Nk = k0*A0*I0
#             for a, a0 in enumerate(a_list):
#                 Na = a0*I0
#                 for m, maxDet in enumerate(m_list):
#                     E = [self.evalImgs[Nk + Na + i] for i in i_list]
#                     E = [e for e in E if not e is None]
#                     if len(E) == 0:
#                         continue
#                     dtScores = np.concatenate([e['dtScores'][0:maxDet] for e in E])

#                     # different sorting method generates slightly different results.
#                     # mergesort is used to be consistent as Matlab implementation.
#                     inds = np.argsort(-dtScores, kind='mergesort')
#                     dtScoresSorted = dtScores[inds]

#                     dtm  = np.concatenate([e['dtMatches'][:,0:maxDet] for e in E], axis=1)[:,inds]
#                     dtIg = np.concatenate([e['dtIgnore'][:,0:maxDet]  for e in E], axis=1)[:,inds]
#                     gtIg = np.concatenate([e['gtIgnore'] for e in E])
#                     npig = np.count_nonzero(gtIg==0 )
#                     if npig == 0:
#                         continue
#                     tps = np.logical_and(               dtm,  np.logical_not(dtIg) )
#                     fps = np.logical_and(np.logical_not(dtm), np.logical_not(dtIg) )

#                     tp_sum = np.cumsum(tps, axis=1).astype(dtype=np.float64)
#                     fp_sum = np.cumsum(fps, axis=1).astype(dtype=np.float64)
#                     for t, (tp, fp) in enumerate(zip(tp_sum, fp_sum)):
#                         tp = np.array(tp)
#                         fp = np.array(fp)
#                         nd = len(tp)
#                         rc = tp / npig
#                         pr = tp / (fp+tp+np.spacing(1))
#                         q  = np.zeros((R,))
#                         ss = np.zeros((R,))

#                         if nd:
#                             recall[t,k,a,m] = rc[-1]
#                         else:
#                             recall[t,k,a,m] = 0

#                         # numpy is slow without cython optimization for accessing elements
#                         # use python array gets significant speed improvement
#                         pr = pr.tolist(); q = q.tolist()

#                         for i in range(nd-1, 0, -1):
#                             if pr[i] > pr[i-1]:
#                                 pr[i-1] = pr[i]

#                         inds = np.searchsorted(rc, p.recThrs, side='left')
#                         try:
#                             for ri, pi in enumerate(inds):
#                                 q[ri] = pr[pi]
#                                 ss[ri] = dtScoresSorted[pi]
#                         except:
#                             pass
#                         precision[t,:,k,a,m] = np.array(q)
#                         scores[t,:,k,a,m] = np.array(ss)
#         self.eval = {
#             'params': p,
#             'counts': [T, R, K, A, M],
#             'date': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
#             'precision': precision,
#             'recall':   recall,
#             'scores': scores,
#         }
#         toc = time.time()
#         print('DONE (t={:0.2f}s).'.format( toc-tic))

#     def summarize(self):
#         \'''
#         Compute and display summary metrics for evaluation results.
#         Note this functin can *only* be applied on the default parameter setting
#         \'''
#         def _summarize( ap=1, iouThr=None, areaRng='all', maxDets=100 ):
#             p = self.params
#             iStr = ' {:<18} {} @[ IoU={:<9} | area={:>6s} | maxDets={:>3d} ] = {:0.3f}'
#             titleStr = 'Average Precision' if ap == 1 else 'Average Recall'
#             typeStr = '(AP)' if ap==1 else '(AR)'
#             iouStr = '{:0.2f}:{:0.2f}'.format(p.iouThrs[0], p.iouThrs[-1]) \
#                 if iouThr is None else '{:0.2f}'.format(iouThr)

#             aind = [i for i, aRng in enumerate(p.areaRngLbl) if aRng == areaRng]
#             mind = [i for i, mDet in enumerate(p.maxDets) if mDet == maxDets]
#             if ap == 1:
#                 # dimension of precision: [TxRxKxAxM]
#                 s = self.eval['precision']
#                 # IoU
#                 if iouThr is not None:
#                     t = np.where(iouThr == p.iouThrs)[0]
#                     s = s[t]
#                 s = s[:,:,:,aind,mind]
#             else:
#                 # dimension of recall: [TxKxAxM]
#                 s = self.eval['recall']
#                 if iouThr is not None:
#                     t = np.where(iouThr == p.iouThrs)[0]
#                     s = s[t]
#                 s = s[:,:,aind,mind]
#             if len(s[s>-1])==0:
#                 mean_s = -1
#             else:
#                 mean_s = np.mean(s[s>-1])
#             print(iStr.format(titleStr, typeStr, iouStr, areaRng, maxDets, mean_s))
#             return mean_s
#         def _summarizeDets():
#             stats = np.zeros((12,))
#             stats[0] = _summarize(1)
#             stats[1] = _summarize(1, iouThr=.5, maxDets=self.params.maxDets[2])
#             stats[2] = _summarize(1, iouThr=.75, maxDets=self.params.maxDets[2])
#             stats[3] = _summarize(1, areaRng='small', maxDets=self.params.maxDets[2])
#             stats[4] = _summarize(1, areaRng='medium', maxDets=self.params.maxDets[2])
#             stats[5] = _summarize(1, areaRng='large', maxDets=self.params.maxDets[2])
#             stats[6] = _summarize(0, maxDets=self.params.maxDets[0])
#             stats[7] = _summarize(0, maxDets=self.params.maxDets[1])
#             stats[8] = _summarize(0, maxDets=self.params.maxDets[2])
#             stats[9] = _summarize(0, areaRng='small', maxDets=self.params.maxDets[2])
#             stats[10] = _summarize(0, areaRng='medium', maxDets=self.params.maxDets[2])
#             stats[11] = _summarize(0, areaRng='large', maxDets=self.params.maxDets[2])
#             return stats
#         def _summarizeKps():
#             stats = np.zeros((10,))
#             stats[0] = _summarize(1, maxDets=20)
#             stats[1] = _summarize(1, maxDets=20, iouThr=.5)
#             stats[2] = _summarize(1, maxDets=20, iouThr=.75)
#             stats[3] = _summarize(1, maxDets=20, areaRng='medium')
#             stats[4] = _summarize(1, maxDets=20, areaRng='large')
#             stats[5] = _summarize(0, maxDets=20)
#             stats[6] = _summarize(0, maxDets=20, iouThr=.5)
#             stats[7] = _summarize(0, maxDets=20, iouThr=.75)
#             stats[8] = _summarize(0, maxDets=20, areaRng='medium')
#             stats[9] = _summarize(0, maxDets=20, areaRng='large')
#             return stats
#         if not self.eval:
#             raise Exception('Please run accumulate() first')
#         iouType = self.params.iouType
#         if iouType == 'segm' or iouType == 'bbox':
#             summarize = _summarizeDets
#         elif iouType == 'keypoints':
#             summarize = _summarizeKps
#         self.stats = summarize()

#     def __str__(self):
#         self.summarize()

# class Params:
#     \'''
#     Params for coco evaluation api
#     \'''
#     def setDetParams(self):
#         self.imgIds = []
#         self.catIds = []
#         # np.arange causes trouble.  the data point on arange is slightly larger than the true value
#         self.iouThrs = np.linspace(.5, 0.95, int(np.round((0.95 - .5) / .05)) + 1, endpoint=True)
#         self.recThrs = np.linspace(.0, 1.00, int(np.round((1.00 - .0) / .01)) + 1, endpoint=True)
#         self.maxDets = [1, 10, 100]
#         self.areaRng = [[0 ** 2, 1e5 ** 2], [0 ** 2, 32 ** 2], [32 ** 2, 96 ** 2], [96 ** 2, 1e5 ** 2]]
#         self.areaRngLbl = ['all', 'small', 'medium', 'large']
#         self.useCats = 1

#     def setKpParams(self):
#         self.imgIds = []
#         self.catIds = []
#         # np.arange causes trouble.  the data point on arange is slightly larger than the true value
#         self.iouThrs = np.linspace(.5, 0.95, int(np.round((0.95 - .5) / .05)) + 1, endpoint=True)
#         self.recThrs = np.linspace(.0, 1.00, int(np.round((1.00 - .0) / .01)) + 1, endpoint=True)
#         self.maxDets = [20]
#         self.areaRng = [[0 ** 2, 1e5 ** 2], [32 ** 2, 96 ** 2], [96 ** 2, 1e5 ** 2]]
#         self.areaRngLbl = ['all', 'medium', 'large']
#         self.useCats = 1
#         self.kpt_oks_sigmas = np.array([.26, .25, .25, .35, .35, .79, .79, .72, .72, .62,.62, 1.07, 1.07, .87, .87, .89, .89])/10.0

#     def __init__(self, iouType='segm'):
#         if iouType == 'segm' or iouType == 'bbox':
#             self.setDetParams()
#         elif iouType == 'keypoints':
#             self.setKpParams()
#         else:
#             raise Exception('iouType not supported')
#         self.iouType = iouType
#         # useSegm is deprecated
#         self.useSegm = None
# ''' 

In [ ]:
# with open('/opt/conda/lib/python3.10/site-packages/pycocotools/cocoeval.py','w') as f:    
#     f.write(update_numpy)

In [ ]:
# def drop_low_confidence(orig_prediction, conf_thresh=0.1):
#     """ Dropping bboxes with low confidence scores
#     """
#     keep = []
#     for idx, conf in enumerate(orig_prediction['scores']):
#         if conf > conf_thresh:
#             keep.append(idx)
#     high_conf_preds = orig_prediction
#     high_conf_preds['boxes'] = high_conf_preds['boxes'][keep]
#     high_conf_preds['scores'] = high_conf_preds['scores'][keep]
#     high_conf_preds['labels'] = high_conf_preds['labels'][keep]
    
#     return high_conf_preds

# def apply_nms(orig_prediction, iou_thresh=0.1):
#     """Non-maximum supression:
#        deleting lower-confidence bboxes overlapped with iou > iou_thres
#     """
#     keep = torchvision.ops.nms(
#         orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
#     final_prediction = orig_prediction
#     final_prediction['boxes'] = final_prediction['boxes'][keep]
#     final_prediction['scores'] = final_prediction['scores'][keep]
#     final_prediction['labels'] = final_prediction['labels'][keep]
    
#     return final_prediction

In [ ]:
# def compute_mAP(model, dataloader):
#     """Computing the mean average precision over a dataset
#     """
#     total_preds = []
#     total_grtruths = []
#     for images, targets in tqdm(dataloader):
#         images = list(image.to(device) for image in images)
#         targets = list(targets)
#         total_grtruths.extend(targets)
#         model = model.to(device)
#         model.eval()
#         with torch.no_grad():
#             preds = model(images)            
#         preds = list({key: val.to('cpu') for key, val in pred.items()} 
#                        for pred in preds)
#         high_conf_preds = list(drop_low_confidence(pred) for pred in preds)
#         final_preds = list(apply_nms(pred) for pred in high_conf_preds)
#         total_preds.extend(final_preds)
# #         total_preds.extend(preds)
        
        
        
#     mAP = MeanAveragePrecision(class_metrics=True, box_format = 'xyxy')
# #      MeanAveragePercision: iou_thresholds (Optional[List[float]]) – IoU thresholds for evaluation. If set to None it corresponds to the stepped range [0.5,...,0.95] with step 0.05
#     mAP.update(total_preds, total_grtruths)
#     return mAP.compute()

In [ ]:
# !pip install torchmetrics[detection]

In [ ]:
# from pprint import pprint
# mAP = compute_mAP(model, valid_data_loader)
# pprint(mAP)

In [ ]:
# ! git clone https://github.com/pytorch/vision.git
# ! cd vision;cp references/detection/utils.py ../;cp references/detection/transforms.py ../;cp references/detection/coco_eval.py ../;cp references/detection/engine.py ../;cp references/detection/coco_utils.py ../

In [ ]:
# from engine import train_one_epoch, evaluate
# import utils

In [ ]:
# fix_bug = '''
# import math
# import sys
# import time

# import torch
# import torchvision.models.detection.mask_rcnn
# import utils
# from coco_eval import CocoEvaluator
# from coco_utils import get_coco_api_from_dataset


# def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
#     model.train()
#     metric_logger = utils.MetricLogger(delimiter="  ")
#     metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
#     header = f"Epoch: [{epoch}]"

#     lr_scheduler = None
#     if epoch == 0:
#         warmup_factor = 1.0 / 1000
#         warmup_iters = min(1000, len(data_loader) - 1)

#         lr_scheduler = torch.optim.lr_scheduler.LinearLR(
#             optimizer, start_factor=warmup_factor, total_iters=warmup_iters
#         )

#     for images, targets in metric_logger.log_every(data_loader, print_freq, header):
#         images = list(image.to(device) for image in images)
#         targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
#         with torch.cuda.amp.autocast(enabled=scaler is not None):
#             loss_dict = model(images, targets)
#             losses = sum(loss for loss in loss_dict.values())

#         # reduce losses over all GPUs for logging purposes
#         loss_dict_reduced = utils.reduce_dict(loss_dict)
#         losses_reduced = sum(loss for loss in loss_dict_reduced.values())

#         loss_value = losses_reduced.item()

#         if not math.isfinite(loss_value):
#             print(f"Loss is {loss_value}, stopping training")
#             print(loss_dict_reduced)
#             sys.exit(1)

#         optimizer.zero_grad()
#         if scaler is not None:
#             scaler.scale(losses).backward()
#             scaler.step(optimizer)
#             scaler.update()
#         else:
#             losses.backward()
#             optimizer.step()

#         if lr_scheduler is not None:
#             lr_scheduler.step()

#         metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
#         metric_logger.update(lr=optimizer.param_groups[0]["lr"])

#     return metric_logger


# def _get_iou_types(model):
#     model_without_ddp = model
#     if isinstance(model, torch.nn.parallel.DistributedDataParallel):
#         model_without_ddp = model.module
#     iou_types = ["bbox"]
#     if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
#         iou_types.append("segm")
#     if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
#         iou_types.append("keypoints")
#     return iou_types


# @torch.inference_mode()
# def evaluate(model, data_loader, device):
#     n_threads = torch.get_num_threads()
#     # FIXME remove this and make paste_masks_in_image run on the GPU
#     torch.set_num_threads(1)
#     cpu_device = torch.device("cpu")
#     model.eval()
#     metric_logger = utils.MetricLogger(delimiter="  ")
#     header = "Test:"

#     coco = get_coco_api_from_dataset(data_loader.dataset)
#     iou_types = _get_iou_types(model)
#     coco_evaluator = CocoEvaluator(coco, iou_types)

#     for images, targets in metric_logger.log_every(data_loader, 100, header):
#         images = list(img.to(device) for img in images)

#         if torch.cuda.is_available():
#             torch.cuda.synchronize()
#         model_time = time.time()
#         outputs = model(images)

#         outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
#         model_time = time.time() - model_time

#         res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
#         evaluator_time = time.time()
#         coco_evaluator.update(res)
#         evaluator_time = time.time() - evaluator_time
#         metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

#     # gather the stats from all processes
#     metric_logger.synchronize_between_processes()
#     print("Averaged stats:", metric_logger)
#     coco_evaluator.synchronize_between_processes()

#     # accumulate predictions from all images
#     coco_evaluator.accumulate()
#     coco_evaluator.summarize()
#     torch.set_num_threads(n_threads)
#     return coco_evaluator
# '''


In [ ]:
# !cat /kaggle/working/engine.py

In [ ]:
# with open('/kaggle/working/engine.py','w') as f:    
#     f.write(fix_bug)

In [ ]:
# !cat /kaggle/working/engine.py

In [ ]:
# evaluate(model, valid_data_loader, device=device)